<a href="https://colab.research.google.com/github/sebac820/tarea-1-recsys/blob/main/Tarea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1

## Librerías

In [ ]:
import pandas as pd

## Actividad 1: Análisis exploratorio

Importamos los datos: